In [ ]:
import pandas as pd
import os

# Load base cleaned CSV (after deduplication)
main_path = "Final_ResearchOutputs_Cleaned.csv"
main_df = pd.read_csv(main_path, dtype=str)
print(f"Loaded base file with {main_df.shape[0]} records.")

In [ ]:
# Load project metadata for enrichment
metadata_path = "ProjectsAllMetadata.xlsx"
metadata_df = pd.read_excel(metadata_path, dtype=str)
print(f"Loaded metadata file with {metadata_df.shape[0]} entries.")

In [ ]:
# Clean column names
main_df.columns = main_df.columns.str.strip()
metadata_df.columns = metadata_df.columns.str.strip()

In [ ]:
# Merge on ProjID and enrich missing fields
if 'ProjID' in main_df.columns and 'ProjID' in metadata_df.columns:
    merged_df = pd.merge(main_df, metadata_df, on='ProjID', how='left', suffixes=('', '_meta'))

    enrichment_fields = [
        'ProjectStatus', 'ProjectTitle', 'ProjectRDC',
        'ProjectYearStarted', 'ProjectYearEnded', 'ProjectPI'
    ]

    for field in enrichment_fields:
        merged_df[field] = merged_df[field].fillna(merged_df[field + '_meta']).replace("", merged_df[field + '_meta'])

    merged_df.drop(columns=[f + '_meta' for f in enrichment_fields if f + '_meta' in merged_df.columns], inplace=True)
else:
    merged_df = main_df.copy()


In [ ]:
# Filter out records that are unlikely to be FSRDC-related
def is_relevant(title, rdc):
    title = str(title).lower()
    rdc = str(rdc).lower()
    return 'fsrdc' in title or 'fsrdc' in rdc

merged_df['is_relevant'] = merged_df.apply(lambda row: is_relevant(row.get('ProjectTitle', ''), row.get('ProjectRDC', '')), axis=1)
filtered_df = merged_df[merged_df['is_relevant'] == True].drop(columns=['is_relevant'])
print(f"Filtered dataset now contains {filtered_df.shape[0]} relevant records.")

In [ ]:
# Ensure all required columns exist
required_columns = [
    'ProjID', 'ProjectStatus', 'ProjectTitle', 'ProjectRDC', 'ProjectYearStarted',
    'ProjectYearEnded', 'ProjectPI', 'OutputTitle', 'OutputBiblio', 'OutputType',
    'OutputStatus', 'OutputVenue', 'OutputYear', 'OutputMonth', 'OutputVolume',
    'OutputNumber', 'OutputPages'
]

for col in required_columns:
    if col not in filtered_df.columns:
        filtered_df[col] = ""

# Fill common empty fields
filtered_df['ProjectStatus'] = filtered_df['ProjectStatus'].replace("", "Completed")
filtered_df['OutputStatus'] = filtered_df['OutputStatus'].replace("", "Published")
filtered_df['OutputType'] = filtered_df['OutputType'].replace("", "Unknown")

# Parse OutputYear and OutputMonth to standard numeric format
filtered_df['OutputYear'] = pd.to_datetime(filtered_df['OutputYear'], errors='coerce').dt.year
filtered_df['OutputMonth'] = pd.to_datetime(filtered_df['OutputMonth'], errors='coerce').dt.month

# Trim strings
filtered_df = filtered_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# Save the enriched file
os.makedirs('./data/processed', exist_ok=True)
output_path = './data/processed/ResearchOutputs_GroupX_Task2_Enriched.csv'
filtered_df.to_csv(output_path, index=False)
print(f"Enriched data saved to {output_path}")